In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px

In [2]:
array_capital_cost =[*range(300,950,50)]
array_lifetime_years = [10, 15, 20]
array_FOM_percent = [2, 3, 4, 5, 6, 7, 8, 9, 10]
array_VOM = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10]
array_carbon_price = [*range(15, 125, 5)]

In [3]:
state_cases = ['CA', 'ID', 'NY', 'TX', 'WA', 'WI', 'WA_Grant_County', 'WI_WE_Energies']

capital_cases = []
for i in array_capital_cost: capital_cases.append('capital_cost_'+str(i))

lifetime_cases = []
for i in array_lifetime_years: lifetime_cases.append('lifetime_years_'+str(i))

fom_cases = []
for i in array_FOM_percent: fom_cases.append('FOM_pct_'+str(i))

vom_cases = []
for i in array_VOM: vom_cases.append('VOM_pct_'+str(i))

carbon_cases = []
for i in array_carbon_price: carbon_cases.append('carbon_price_'+str(i))


In [4]:
files_to_read = ['base.csv', 'base_refrigerant.csv']

for i in state_cases: files_to_read.append(i+'.csv')
for i in capital_cases: files_to_read.append(i+'.csv')
for i in lifetime_cases: files_to_read.append(i+'.csv')
for i in fom_cases: files_to_read.append(i+'.csv')
for i in vom_cases: files_to_read.append(i+'.csv')
for i in carbon_cases: files_to_read.append(i+'.csv')

# Getting Columns from the base file
df_input = pd.read_csv('output/base.csv')
columns = ['filenames']
for i in df_input['Variable']: columns.append(i)

df = pd.DataFrame(0.0, index = range(len(files_to_read)), columns = columns)
print(df_input)

    Unnamed: 0                                Variable               Value
0            0          Cold Temperature Available (C)                  45
1            1              Cold Temperature Final (C)               43.92
2            2               Cold Mass Flowrate (kg/s)               100.0
3            3             Hot Temperature Desired (C)                  85
4            4             Hot Temperature Minimum (C)                  80
5            5                Hot Mass Flowrate (kg/s)               0.025
6            6                    Ideal COP Calculated                8.95
7            7                    Selected Refrigerant              R245ca
8            8         Estimated Compressor Efficiency               0.635
9            9                Carnot Efficiency Factor                0.45
10          10           Carnot Efficiency Factor Flag                True
11          11                   Actual COP Calculated                4.03
12          12           

In [5]:
df['filenames'] = files_to_read
for i in df.index:
    #print(df.loc[i,'filenames'])
    df_input = pd.read_csv('output/'+df.loc[i,'filenames'])
    for j in df_input.index:
        df.loc[i,df_input.loc[j,'Variable']] = df_input.loc[j,'Value']

df.to_excel('data.xlsx')

In [12]:
#print(df[10:23]['HP Capital Cost Per Unit ($/MMBTU/hr)'][:])
#print(df[10:23]['Net Present Value'][:])

df['Cost of Carbon ($/ton)'] = 0.0
for i in df.index:
    df.loc[i,'Cost of Carbon ($/ton)'] = float(df.loc[i,'Gas Social Cost of Emissions'])/float(df.loc[i,'Gas Emissions'])

fig = px.line(df[10:23], x="HP Capital Cost Per Unit ($/MMBTU/hr)", y="Net Present Value", title='Net Present Value vs. Capital Cost')
fig.update_layout(template = 'simple_white')
fig.write_image('images/capital_cost.png')
fig.show()

fig = px.line(df[23:26], x="Project Lifetime", y="Net Present Value", title='Net Present Value vs. Project Lifetime')
fig.update_layout(template = 'simple_white')
fig.write_image('images/lifetime.png')
fig.show()

fig = px.line(df[26:35], x="HP Fixed O&M Costs ($/MMBTU/hr/yr)", y="Net Present Value", title='Net Present Value vs. FO&M Costs')
fig.update_layout(template = 'simple_white')
fig.write_image('images/FOM.png')
fig.show()

fig = px.line(df[35:45], x="HP Variable O&M Costs ($/MMBTU)", y="Net Present Value", title='Net Present Value vs. VO&M Costs')
fig.update_layout(template = 'simple_white')
fig.write_image('images/VOM.png')
fig.show()

fig = px.line(df[45:], x='Cost of Carbon ($/ton)', y="Net Present Value", title='Net Present Value vs. Carbon Cost')
fig.update_layout(template = 'simple_white')
fig.write_image('images/carbon_cost.png')
fig.show()


In [17]:
df_break_even = pd.read_excel('output/break_even.xlsx')
df_break_even.dropna(axis = 0, inplace= True)
print(df_break_even)

       Unnamed: 0  Capital Cost  Gas_Price  Electricity_Price  HP_LCOH  \
6               6           300        1.6              0.010    0.151   
7               7           300        1.7              0.010    0.151   
8               8           300        1.8              0.010    0.151   
9               9           300        1.9              0.010    0.151   
10             10           300        2.0              0.010    0.151   
...           ...           ...        ...                ...      ...   
51050       51050           900       10.0              0.170    0.636   
51140       51140           900        9.9              0.171    0.639   
51141       51141           900       10.0              0.171    0.639   
51232       51232           900       10.0              0.172    0.642   
51323       51323           900       10.0              0.173    0.645   

       Gas_LCOH        NPV   IRR        PBP  
6         0.080 -374773.17 -57.9 -27.989891  
7         0.084 -35

In [35]:
capital_costs = [300, 600, 900]
#capital_cost = 300
for capital_cost in capital_costs:
    mask = df_break_even['Capital Cost' ] == capital_cost
    x_solved = []
    y_solved = []
    z_solved = []

    #print(df_break_even)
    for i in df_break_even[mask].index:
        if (df_break_even.loc[i,'PBP'] > 0.0) & (df_break_even.loc[i,'PBP'] < 20.1
        ):
            x_solved.append(df_break_even.loc[i,'Gas_Price'])
            y_solved.append(df_break_even.loc[i,'Electricity_Price'])
            z_solved.append(df_break_even.loc[i,'PBP'])


    fig = go.Figure(data=go.Scatter(
        x = x_solved,
        y = y_solved,
        mode='markers',
        marker=dict(
            size=16,
            color=z_solved, #set color equal to a variable
            colorscale='Jet', # one of plotly colorscales
            showscale=True
        )
    ))
    fig.update_traces(mode='markers', marker_line_width=0, marker_size=9)
    fig.update_xaxes(title_text = 'Gas Price (€/MMBTU)')
    fig.update_yaxes(title_text = 'Electricity Price (€/kWh)')
    fig.update_layout(template = 'simple_white', width = 600, height = 800, title = '$'+str(capital_cost)+'/kW Capital Cost')
    fig.write_image('images/'+str(capital_cost) + '_capital_cost.png')
    fig.show()